# Documentation pages

## List of repos

In [ ]:
def get_repo_stubs_from_github(org='i2mint'):
    import hubcap
    org_reader = hubcap.GitHubReader(org)
    return [f"{org}/{repo}" for repo in list(org_reader)]

def get_repo_stubs(savefile=None, org='i2mint'):
    import json
    import os

    if not savefile.endswith('.json'):
        org = savefile
        savefile = f'{org}_repo_stubs.json'

    if os.path.exists(savefile):
        with open(savefile, 'r') as f:
            repo_stubs = json.load(f)
    else:
        repo_stubs = get_repo_stubs_from_github()
        with open(savefile, 'w') as f:
            json.dump(repo_stubs, f)

    return repo_stubs



In [51]:
# repo_stubs = [f"{org}/{repo}" for repo in list(org_reader)]

repo_stubs = get_repo_stubs('i2mint')
len(repo_stubs)

107

## Check what repos don't have documentation pages

In [70]:
def documentation_exists(
    repo: str,
    repo_to_documentation_url = "https://{}.github.io/{}".format,
    org=None
):
    import requests 

    if '/' in repo:
        org, repo = repo.split('/')
    else:
        assert org is not None, 'If repo does not contain org, org must be provided'
    
    url = repo_to_documentation_url(org, repo)
    r = requests.get(url)
    return r.status_code == 200

def gather_docs_existence(repo_stubs):
    for repo in repo_stubs:
        try:
            yield {'repo': repo, 'docs': documentation_exists(repo)}
        except Exception as e:
            yield {'repo': repo, 'docs': False, 'error': str(e)}

In [74]:
import pandas as pd

docs_exist_df = pd.DataFrame(list(gather_docs_existence(repo_stubs)))
print(f"{docs_exist_df.shape=}")

docs_exist_df.head()


docs_exist_df.shape=(107, 2)


,repo,docs
0,i2mint/aiofiledol,True
1,i2mint/arangodol,True
2,i2mint/atypes,False
3,i2mint/audiostream2py,True
4,i2mint/azuredol,True


In [75]:
docs_exist_df['docs'].value_counts()

docs
True     96
False    11
Name: count, dtype: int64

In [76]:
docs_exist_df[docs_exist_df['docs'] == False]

,repo,docs
2,i2mint/atypes,False
6,i2mint/chromadol,False
8,i2mint/content,False
17,i2mint/dol_cookbook,False
23,i2mint/examples,False
37,i2mint/i2i_scrap,False
61,i2mint/pdfdol,False
71,i2mint/py2misc,False
80,i2mint/redisposted,False
82,i2mint/s3dol,False


In [78]:
# reverify just the ones that failed

retry_stubs = docs_exist_df[docs_exist_df['docs'] == False]['repo']

retry_df = pd.DataFrame(list(gather_docs_existence(retry_stubs)))

now_true = retry_df[retry_df['docs'] == True]
still_false = retry_df[retry_df['docs'] == False]

print(f"{now_true.shape=}")
print(f"{still_false.shape=}")
still_false

now_true.shape=(4, 2)
still_false.shape=(7, 2)


,repo,docs
2,i2mint/content,False
3,i2mint/dol_cookbook,False
4,i2mint/examples,False
5,i2mint/i2i_scrap,False
7,i2mint/py2misc,False
9,i2mint/s3dol,False
10,i2mint/spearhead_sump,False


In [82]:
print(still_false.reset_index(drop=True).to_markdown())

|    | repo                  | docs   |
|---:|:----------------------|:-------|
|  0 | i2mint/content        | False  |
|  1 | i2mint/dol_cookbook   | False  |
|  2 | i2mint/examples       | False  |
|  3 | i2mint/i2i_scrap      | False  |
|  4 | i2mint/py2misc        | False  |
|  5 | i2mint/s3dol          | False  |
|  6 | i2mint/spearhead_sump | False  |


In [86]:
import lkj

lkj.wrapped_print(still_false.repo.to_list(), max_width=120)

i2mint/content, i2mint/dol_cookbook, i2mint/examples, i2mint/i2i_scrap, i2mint/py2misc, i2mint/s3dol,
i2mint/spearhead_sump


## Make gh-pages branch

In [43]:
from epythet.tools.published_docs import ensure_branch

for i, repo_stub in enumerate(repo_stubs):
    ensure_branch(repo_stub, branch='gh-pages')

i2mint/py2dash gh-pages branch doesn't exist: 404, response.content=b'{"message":"Branch not found","documentation_url":"https://docs.github.com/rest/branches/branches#get-a-branch","status":"404"}'
i2mint/py2dash Branch gh-pages has been successfully created.
i2mint/py2misc gh-pages branch doesn't exist: 404, response.content=b'{"message":"Branch not found","documentation_url":"https://docs.github.com/rest/branches/branches#get-a-branch","status":"404"}'
i2mint/py2misc Branch gh-pages has been successfully created.
i2mint/py2mqtt gh-pages branch doesn't exist: 404, response.content=b'{"message":"Branch not found","documentation_url":"https://docs.github.com/rest/branches/branches#get-a-branch","status":"404"}'
i2mint/py2mqtt Branch gh-pages has been successfully created.
i2mint/py2store gh-pages branch doesn't exist: 404, response.content=b'{"message":"Branch not found","documentation_url":"https://docs.github.com/rest/branches/branches#get-a-branch","status":"404"}'
i2mint/py2store B

## Set github settings for pages (no working)

In [ ]:
from epythet.tools.published_docs import configure_github_pages

In [44]:
# TODO: Doesn't seem to work. Repair, or leave settings to be done manually?
# def configure_github_pages_for_stubs(repo_stubs):
#     for repo_stub in repo_stubs:
#         yield repo_stub, configure_github_pages(repo_stub)

# repo_pages_status = dict(configure_github_pages_for_stubs(repo_stubs))

## Analyze repo_pages_status

In [ ]:
repo_pages_status['i2mint/redisposted'].content

b'{"message":"Not Found","documentation_url":"https://docs.github.com/rest/pages/pages#update-information-about-a-apiname-pages-site","status":"404"}'

In [ ]:
import pickle

f = '/Users/thorwhalen/Dropbox/tmp/i2mint_docs_status.pkl'
pickle.dump(repo_pages_status, open(f, 'wb'))

In [ ]:
# None value means that the process was successful
successful = [k for k, v in repo_pages_status.items() if v is None]
print(f"{len(successful)} out of {len(repo_pages_status)} repos have been successfully configured for github pages")
successful

16 out of 107 repos have been successfully configured for github pages


['i2mint/audiostream2py',
 'i2mint/azuredol',
 'i2mint/config2py',
 'i2mint/dol',
 'i2mint/epythet',
 'i2mint/i2',
 'i2mint/isee',
 'i2mint/meshed',
 'i2mint/mongodol',
 'i2mint/pdfdol',
 'i2mint/posted',
 'i2mint/py2http',
 'i2mint/py2json',
 'i2mint/s3dol',
 'i2mint/tabled',
 'i2mint/wads']

In [ ]:
d = {
    'ok': list(),
    'missing_branch': list(),
    'other': list()
}

for k, v in repo_pages_status.items():
    if v is None:
        d['ok'].append(k)
    elif isinstance(v, str):
        d['missing_branch'].append(k)
    else:
        d['other'].append(k)

counts = {k: len(v) for k, v in d.items()}
counts

{'ok': 16, 'missing_branch': 81, 'other': 10}

In [ ]:
print('\n'.join(f"https://github.com/{x}" for x in d['other']))

https://github.com/i2mint/chromadol
https://github.com/i2mint/dol_cookbook
https://github.com/i2mint/doodad
https://github.com/i2mint/ju
https://github.com/i2mint/kafkaposted
https://github.com/i2mint/lkj
https://github.com/i2mint/redisposted
https://github.com/i2mint/sqldol
https://github.com/i2mint/titbit
https://github.com/i2mint/xdol


In [ ]:
missing_branch_urls = [f"https://github.com/{x}" for x in d['missing_branch']]
missing_branch_urls[:10]

['https://github.com/i2mint/aiofiledol',
 'https://github.com/i2mint/arangodol',
 'https://github.com/i2mint/atypes',
 'https://github.com/i2mint/c2py',
 'https://github.com/i2mint/content',
 'https://github.com/i2mint/couchdol',
 'https://github.com/i2mint/creek',
 'https://github.com/i2mint/crude',
 'https://github.com/i2mint/dacc',
 'https://github.com/i2mint/dagapp']

In [ ]:
# repo_pages_status

## Misc github pokes

In [ ]:
from epythet.tools.published_docs import (
    default_branch_and_commit_sha, repo_data, commit_data, ensure_branch
)


In [11]:
repo_stub = 'i2mint/kafkaposted'

In [13]:
r = repo_data(repo_stub)
print(f"{len(r)} fields in the repo_data for {repo_stub}.\n")

from lkj import wrapped_print

wrapped_print(r)



97 fields in the repo_data for i2mint/kafkaposted.

id, node_id, name, full_name, private, owner, html_url, description, fork, url,
forks_url, keys_url, collaborators_url, teams_url, hooks_url, issue_events_url,
events_url, assignees_url, branches_url, tags_url, blobs_url, git_tags_url,
git_refs_url, trees_url, statuses_url, languages_url, stargazers_url,
contributors_url, subscribers_url, subscription_url, commits_url,
git_commits_url, comments_url, issue_comment_url, contents_url, compare_url,
merges_url, archive_url, downloads_url, issues_url, pulls_url, milestones_url,
notifications_url, labels_url, releases_url, deployments_url, created_at,
updated_at, pushed_at, git_url, ssh_url, clone_url, svn_url, homepage, size,
stargazers_count, watchers_count, language, has_issues, has_projects,
has_downloads, has_wiki, has_pages, has_discussions, forks_count, mirror_url,
archived, disabled, open_issues_count, license, allow_forking, is_template,
web_commit_signoff_required, topics, visibili

In [14]:
default_branch_and_commit_sha(repo_stub)

{'default_branch': 'master',
 'commit_sha': '0689485961069bef4a24847d8b088d6fcd443960'}

# isee get_new_version

In [ ]:
from isee.generation_utils import get_new_version


t = get_new_version(
    work_tree='/Users/thorwhalen/Dropbox/py/proj/i/tabled',
    version_patch_prefix='vvv'
)
t



'0.1.vvv11'

In [ ]:
"GEN AI- TOO MUCH SPEND, TOO LITTLE BENEFIT".lower()

'gen ai- too much spend, too little benefit'